# HW1-2
## ID3 - Part 1
### Fatemeh Rafiee, Reyhaneh Ahani

In [1]:
import math
import numpy as np
import pandas as pd
from collections import deque
from sklearn.model_selection import train_test_split

In [2]:
class Node:
    # initialize a separate class for nodes
    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None

class DecisionTree:
    # here we initialize 4 important attribues for DecisionTree
    def __init__(self, X, labels, features):
        self.X = X
        self.features = features
        self.labels = labels
        self.node = None
        
    # calculates the information gain of a given set
    def inf_gain_cal(self, x, feature_idx):
        # calculates entropy of all data
        total_entropy = self.entropy_cal(x)
        x_features, feature_cnt = [], []
        
        # this loop stores all values of each feature in a list
        for i in x:
            x_features.append(self.X[i][feature_idx])
         
        # this loop stores the number of repetition of each value in a list
        for j in list(set(x_features)):
            feature_cnt.append(x_features.count(j))
         
        # in this part samples get separated based on the values of each feature
        feature_v_id = []
        for feature_item in list(set(x_features)):
            tmp = []
            for i, x_i in enumerate(x_features):
                # checks if a specific x has a similar value
                if x_i == feature_item:
                    # stores it in a temporary list
                    tmp.append(x[i])
            feature_v_id.append(tmp) 
            
        # calculates entropy of each group    
        entropy = []
        for n_i, Class in zip(feature_cnt, feature_v_id):
            entropy.append(n_i/len(x)*self.entropy_cal(Class))
            
        return total_entropy - sum(entropy)
        
    # this function calculates the entropy based on its formula    
    def entropy_cal(self, x):
        labels, label_cnt, sigma_entropy = [], [], 0
        
        # at first all labels (targets) of a given set of samples
        # and their repeated times are stores in two lists
        for i in x:
            labels.append(self.labels[i])
            
        for j in list(set(self.labels)):
            label_cnt.append(labels.count(j))

        # for each group of those 
        for cnt in label_cnt:
            if cnt != 0:
                sigma_entropy += -cnt/len(x)*math.log(cnt/len(x), 2)
        return sigma_entropy
    
    def opt_feature(self, x, features):
        # this loops gets the entropy of each feature
        features_entropy = [self.inf_gain_cal(x, feature_id) for feature_id in features]
        # finds the feature with max entropy, then finds the index
        max_id = features[features_entropy.index(max(features_entropy))]
        return self.features[max_id], max_id

    # main part of the tree, recursive function on each node
    def rec(self, x_idx, feature_idx, node):
        # checks whether there's been a node created or not
        if not node:
            node = Node() 
            
        # adds all labels to a temporary list
        labels_tmp = []
        for i in x_idx:
            labels_tmp.append(self.labels[i])
        
        # checks if all the samples of a node are similar (ent = 0) 
        if len(set(labels_tmp)) == 1:
            node.value = self.labels[x_idx[0]]
            return node

        # checks if there're any features left to continiue
        if len(feature_idx) == 0:
            node.value = max(labels_tmp, key=labels_tmp.count) # voting
            return node
        
        # finds the best feature for those specific samples in the node
        best_feature_name, best_feature_idx = self.opt_feature(x_idx, feature_idx)
        node.value = best_feature_name
        node.childs = []
        
        # unique value of the chosen feature for each instance
        unique_values = []
        for x in x_idx:
            unique_values.append(self.X[x][best_feature_idx])
        
        # removes all similar elements 
        unique_values = list(set(unique_values))
    
        for value in unique_values:
            child = Node()
            child.value = value 
            node.childs.append(child) 
           
            # this creates a list of the values of samples for the best feature
            child_idx = []
            for i in x_idx:
                if self.X[i][best_feature_idx] == value:
                    child_idx.append(i)
            
            # checks if there's no sample for that value in a node
            if not child_idx:
                child.next = max(unique_values, key=unique_values.count)
            # otherwise it'll check whether the best feature has already been chosen or not
            else:
                if feature_idx and best_feature_idx in feature_idx: 
                    # if it's the first time it's selected, we remove it from feature_idx list
                    # so that we won't be able to choose it once again 
                    feature_idx.pop(feature_idx.index(best_feature_idx))
                # call the function itself recursivelly 
                child.next = self.rec(child_idx, feature_idx, child.next)   
        return node    

    # this functions fits the model
    def fit(self):
        # at first creates two lists to have the length of X and features
        x_idx = list(range(len(self.X)))
        features_idx = list(range(len(self.features)))
        self.node = self.rec(x_idx, features_idx, self.node)
            
            
    # this predicts the results of the model
    def predict(self, X_test):
        # creates a list with default None values in order to replace them later
        result = [None for _ in range(len(X_test))]
        # start going through each sample
        for i, x_i in enumerate(X_test):
            # creates a class of deque to save nodes
            nodes = deque()
            nodes.append(self.node)
            current_feature = None
            last_child = None
            # we can do these things only if there's at least one one left
            while len(nodes) > 0:
                # eliminate one node from left side
                node = nodes.popleft()
                
                # this checks if the node value is a feature name or not
                for feature in self.features:
                    # if it's the name of a feature it'll store it in current_feature
                    if node.value == feature:
                        current_feature = feature
                        
                # checks if there're any children left
                if node.childs:
                    for child in node.childs:
                        if x_i[self.features.index(current_feature)] == child.value:
                            # stores the value of a feature in a variable (last_child)
                            last_child = child.next.value
                            nodes.append(child.next)
                            
                # if we reach last node it'll evaluate result list with last_child value
                else:
                    result[i] = last_child
                    break
        return result
                    
    # this function calculated the accuracy regarding test data            
    def accuracy(self, X_test, y_test):
        # define a variable to record all misclassified samples
        misclassified = 0
        y_pred = self.predict(X_test)
        # comparing real labels to predicted ones
        for h in range(len(y_pred)):
            if y_pred[h] != y_test[h]:
                # increase this variable if they don't match
                misclassified += 1
                
        return (len(y_pred) - misclassified) / len(y_pred)

In [3]:
df = pd.DataFrame(pd.read_csv("nursery.csv"))
df

,parents,has_nurs,form,children,housing,finance,social,health,final evaluation
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
...,...,...,...,...,...,...,...,...,...
12955,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,priority,spec_prior
12956,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,not_recom,not_recom
12957,great_pret,very_crit,foster,more,critical,inconv,problematic,recommended,spec_prior
12958,great_pret,very_crit,foster,more,critical,inconv,problematic,priority,spec_prior


In [4]:
# df.describe()

In [5]:
df.value_counts('final evaluation')

final evaluation
not_recom     4320
priority      4266
spec_prior    4044
very_recom     328
recommend        2
dtype: int64

In [6]:
df = df.rename(columns={'final evaluation': 'final_evaluation'})

In [7]:
df[df['final_evaluation']=='recommend']

,parents,has_nurs,form,children,housing,finance,social,health,final_evaluation
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend


In [8]:
df.drop(df[(df.final_evaluation == 'recommend')].index)

,parents,has_nurs,form,children,housing,finance,social,health,final_evaluation
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
5,usual,proper,complete,1,convenient,convenient,slightly_prob,not_recom,not_recom
6,usual,proper,complete,1,convenient,convenient,problematic,recommended,priority
...,...,...,...,...,...,...,...,...,...
12955,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,priority,spec_prior
12956,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,not_recom,not_recom
12957,great_pret,very_crit,foster,more,critical,inconv,problematic,recommended,spec_prior
12958,great_pret,very_crit,foster,more,critical,inconv,problematic,priority,spec_prior


In [9]:
X = np.array(df.drop('final_evaluation', axis=1).copy())
y = np.array(df['final_evaluation'].copy())
# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=18, test_size=0.2)

features = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']
dt = DecisionTree(X_train, y_train, features)

In [10]:
dt.fit()
dt.accuracy(X_test, y_test)

0.7993827160493827